Now we will train the Particle Sequence classifier.

To run this notebook we used the following configuration:
* *Software stack*: LCG 94 (it has spark 2.3.1)
* *Platform*: centos7-gcc7
* *Spark cluster*: Hadalytic

In [2]:
# Check if Spark Session has been created correctly
spark

In [3]:
# Add the BDL zip file
sc.addPyFile("/eos/project/s/swan/public/BigDL/bigdl-0.7.0-python-api.zip")

## Load train and test dataset

In [4]:
PATH = "hdfs://hadalytic/project/ML/data/swan/"

trainDF = spark.read.format('parquet')\
        .load(PATH + 'trainUndersampled.parquet')\
        .select(['GRU_input', 'encoded_label'])
        
testDF = spark.read.format('parquet')\
        .load(PATH + 'testUndersampled.parquet')\
        .select(['GRU_input', 'encoded_label'])

In [5]:
trainDF.printSchema()

root
 |-- GRU_input: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- encoded_label: vector (nullable = true)



## Create the model

In [47]:
# Start BigDL
from bigdl.util.common import init_engine
init_engine()

In [58]:
from bigdl.nn.layer import *

recurrent = Recurrent()
recurrent.add(
    GRU(
        input_size=19,
         hidden_size=50,
        activation='tanh'
        )
)

model = Sequential()
model.add(Masking(0.0))
model.add(recurrent)
model.add(Select(2,-1))
model.add(BatchNormalization(50))
model.add(Linear(50,3))
model.add(SoftMax())

creating: createRecurrent
creating: createSigmoid
creating: createTanh
creating: createGRU
creating: createSequential
creating: createMasking
creating: createSelect
creating: createBatchNormalization
creating: createLinear
creating: createSoftMax


## Create train and valiation RDD

We need to create an RDD of `Sample`, a tuple of the form (`features`, `label`). The two elements of this touple should be `numpy arrays`. 

In [60]:
from bigdl.util.common import Sample
import numpy as np

trainRDD = trainDF.rdd.map(lambda row: Sample.from_ndarray(
    np.array(row.GRU_input),
    np.array(row.encoded_label)
))

testRDD = testDF.rdd.map(lambda row: Sample.from_ndarray(
    np.array(row.GRU_input),
    np.array(row.encoded_label)
))

In [11]:
# Let's have a look at one element of trainRDD
trainRDD.take(1)

[Sample: features: [JTensor: storage: [ 2.2783043  -0.38855186 -0.10147543 ... 28.284271    0.
  -1.5777194 ], shape: [801  19], float], labels: [JTensor: storage: [0. 1. 0.], shape: [3], float]]

We can see that `Sample` is composed by the list of 801 particles with 19 features each (`shape=[801 19]`) and the encoded label (`shape=[3]`).

## Optimizer setup and training

In [61]:
# Set of hyperparameters
numEpochs = 50

# The batch used by BDL must be a multiple of numExecutors * executorCores
# Because data will be equally distibuted inside each executor

workerBatch = 64
numExecutors = int(sc._conf.get('spark.executor.instances'))
executorCores = int(sc._conf.get('spark.executor.cores'))

BDLbatch = workerBatch * numExecutors * executorCores

In [62]:
from bigdl.optim.optimizer import *
from bigdl.nn.criterion import CategoricalCrossEntropy

optimizer = Optimizer(
        model = model,
        training_rdd = trainRDD,
        criterion = CategoricalCrossEntropy(),
        optim_method = Adam(),
        end_trigger = MaxEpoch(numEpochs),
        batch_size = BDLbatch   
)

# Optionally you can also set validation
optimizer.set_validation(
    batch_size=BDLbatch,
    val_rdd=testRDD,
    trigger=EveryEpoch(),
    val_method=[Loss(CategoricalCrossEntropy())]
)

creating: createCategoricalCrossEntropy
creating: createAdam
creating: createMaxEpoch
creating: createDistriOptimizer
creating: createEveryEpoch
creating: createCategoricalCrossEntropy
creating: createLoss


Let's define a directory to store logs (i.e. train and validation losses) and save models

In [63]:
# name of our application
appName = "ParticleSequenceClassifier"

# Change it! 
logDir = "/eos/user/m/migliori/SWAN_projects/SparkPipeline/BDLlogs"

# Check if there is already an application with the same name 
# and remove it, otherwise logs will be appended to that app
import os
try:
    os.system('rm -rf '+logDir+'/'+appName)
except:
    pass

print("Saving logs to {}".format(logDir+'/'+appName))

Saving logs to /eos/user/m/migliori/SWAN_projects/SparkPipeline/BDLlogs/ParticleSequenceClassifier


In [64]:
trainSummary = TrainSummary(log_dir=logDir,app_name=appName)
optimizer.set_train_summary(trainSummary)

valSummary = ValidationSummary(log_dir=logDir,app_name=appName)
optimizer.set_val_summary(valSummary)

creating: createTrainSummary
creating: createValidationSummary


We are now ready to launch the training.

Warnign: During the trainign it would be better to shutdown the Toggle Spark Monitorin Display because each iteration is seen as a spark job, therefore the toggle will try to display everything causing problem to the browser. 

In [65]:
%%time 
trained_model = optimizer.optimize()

CPU times: user 13.1 s, sys: 5.32 s, total: 18.5 s
Wall time: 16min 10s


## Plot loss

<IPython.core.display.Javascript object>


In [66]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
%matplotlib notebook

loss = np.array(trainSummary.read_scalar("Loss"))
val_loss = np.array(valSummary.read_scalar("Loss"))

plt.plot(loss[:,0], loss[:,1], label="Training loss")
plt.plot(val_loss[:,0], val_loss[:,1], label="Validation loss", color='crimson', alpha=0.8)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.title("Particle sequence classifier loss")
plt.show()

## Save the model

In [42]:
modelDir = logDir + '/models'
trained_model.saveModel(
            modelPath = modelDir + '/' + appName + '.bigdl',
            weightPath = modelDir + '/' + appName + '.bin',
            over_write = True
)

It is possible to load the model in the following way:
```Python
model = Model.loadModel(modelPath=modelPath+'.bigdl', weightPath=modelPath+'.bin')
```

## Prediction

In [67]:
pred = trained_model.predict(testRDD)

In [68]:
y_pred = np.asarray(pred.collect())
y_true = np.asarray(testDF.select('encoded_label').rdd\
                    .map(lambda row: np.asarray(row.encoded_label)).collect())

In [69]:
from sklearn.metrics import roc_curve, auc
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

<IPython.core.display.Javascript object>


In [70]:
plt.figure()
plt.plot(fpr[0], tpr[0], lw=2, 
         label='HLF classifier (AUC) = %0.4f' % roc_auc[0])
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Background Contamination (FPR)')
plt.ylabel('Signal Efficiency (TPR)')
plt.title('$tt$ selector')
plt.legend(loc="lower right")
plt.show()